In [ ]:
import pathlib
import lzma
import re
import os
import datetime
import copy
import functools

import numpy as np
import pandas as pd

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys._icom.extract
import pymedphys.mudensity
import pymedphys

In [ ]:
patient_id = '008566'

In [ ]:
patients_dir = pathlib.Path(r'\\physics-server\iComLogFiles\patients')
patient_data_paths = list(patients_dir.glob(f'{patient_id}_*/*.xz'))
patient_data_paths

In [ ]:
patient_data_path = patient_data_paths[0]
with lzma.open(patient_data_path, 'r') as f:
    patient_data = f.read()

In [ ]:
DATE_PATTERN = re.compile(rb"\d\d\d\d-\d\d-\d\d\d\d:\d\d:\d\d.")

def get_data_points(data):
    date_index = [m.span() for m in DATE_PATTERN.finditer(data)]
    start_points = [span[0] - 8 for span in date_index]

    end_points = start_points[1::] + [None]

    data_points = [data[start:end] for start, end in zip(start_points, end_points)]
    return data_points

In [ ]:
patient_data_list = get_data_points(patient_data)
len(patient_data_list)

In [ ]:
@functools.lru_cache()
def get_coll_regex(label, number):
    header = rb"0\xb8\x00DS\x00R.\x00\x00\x00" + label + b"\n"
    item = rb"0\x1c\x01DS\x00R.\x00\x00\x00(-?\d+\.\d+)"

    regex = re.compile(header + b"\n".join([item] * number))
    return regex


def extract_coll(data, label, number):
    regex = get_coll_regex(label, number)

    match = regex.search(data)
    span = match.span()

    data = data[0 : span[0]] + data[span[1] + 1 : :]
    items = np.array([float(item) for item in match.groups()])

    return data, items

In [ ]:
def get_delivery_data_items(single_icom_stream):
    shrunk_stream, mu = pymedphys._icom.extract.extract(single_icom_stream, "Delivery MU")
    shrunk_stream, gantry = pymedphys._icom.extract.extract(shrunk_stream, "Gantry")
    shrunk_stream, collimator = pymedphys._icom.extract.extract(shrunk_stream, "Collimator")
    
    shrunk_stream, mlc = extract_coll(shrunk_stream, b"MLCX", 160)
    mlc = mlc.reshape((80,2))
    mlc = np.fliplr(np.flipud(mlc * 10))
    mlc[:,1] = -mlc[:,1]
    mlc = np.round(mlc,10)
#     shrunk_stream, result["ASYMX"] = extract_coll(shrunk_stream, b"ASYMX", 2)
    shrunk_stream, jaw = extract_coll(shrunk_stream, b"ASYMY", 2)
    jaw = np.round(np.array(jaw) * 10, 10)
    jaw = np.flipud(jaw)
    
    return mu, gantry, collimator, mlc, jaw

In [ ]:
mu, gantry, collimator, mlc, jaw = get_delivery_data_items(patient_data_list[250])

In [ ]:
gantry

In [ ]:
collimator

In [ ]:
mu

In [ ]:
jaw

In [ ]:
len(patient_data_list)

In [ ]:
delivery_raw = [
    get_delivery_data_items(single_icom_stream)
    for single_icom_stream in patient_data_list
]

In [ ]:
mu = np.array([item[0] for item in delivery_raw])
diff_mu = np.concatenate([[0], np.diff(mu)])
diff_mu[diff_mu<0] = 0
mu = np.cumsum(diff_mu)

In [ ]:
gantry = np.array([item[1] for item in delivery_raw])
collimator = np.array([item[2] for item in delivery_raw])
mlc = np.array([item[3] for item in delivery_raw])
jaw = np.array([item[4] for item in delivery_raw])

In [ ]:
icom_delivery = pymedphys.Delivery(mu, gantry, collimator, mlc, jaw)
icom_delivery = icom_delivery._filter_cps()

In [ ]:
monaco_directory = pathlib.Path(r'\\monacoda\FocalData\RCCC\1~Clinical')

In [ ]:
tel_path = list(monaco_directory.glob(f'*~{patient_id}/plan/*/tel.1'))[-1]
tel_path

In [ ]:
GRID = pymedphys.mudensity.grid()

delivery_tel = pymedphys.Delivery.from_monaco(tel_path)
mudensity_tel = delivery_tel.mudensity()
pymedphys.mudensity.display(GRID, mudensity_tel)

In [ ]:
mudensity_icom = icom_delivery.mudensity()
pymedphys.mudensity.display(GRID, mudensity_icom)

In [ ]:
icom_delivery.mu[16]

In [ ]:
delivery_tel.mu[1]

In [ ]:
delivery_tel.mlc[1]

In [ ]:
icom_delivery.mlc[16]

In [ ]:
delivery_tel.jaw[1]

In [ ]:
icom_delivery.jaw[16]

In [ ]:
# new_mlc = np.fliplr(np.flipud(np.array(icom_delivery.mlc[16]) * 10))
# new_mlc[:,1] = -new_mlc[:,1]
# new_mlc